In [17]:
from trl import SFTTrainer
import os
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging, GenerationConfig, get_cosine_with_hard_restarts_schedule_with_warmup
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
import torch
import gc
import wandb
from tqdm.notebook import tqdm
import re
import random
from huggingface_hub import login
from transformers.integrations import WandbCallback
from torch.utils.data import DataLoader
import shutil
import json

In [18]:
dataset_name = "evgmaslov/cars"
text2text_model_name = "mistralai/Mistral-Nemo-Instruct-2407"
text2code_model_name = "mistralai/Mistral-Nemo-Instruct-2407"
text2code_tuned_model_name = "evgmaslov/Mistral-Nemo-Instruct-2407-cars"

llama3_format_prompt_inference = "{prompt}<|eot_id|><|start_header_id|>{input}<|end_header_id|>"
llama3_format_noprompt_inference = "<|eot_id|><|start_header_id|>{input}<|end_header_id|>"
llama3_format_prompt_train = "{prompt}<|eot_id|><|start_header_id|>{input}<|end_header_id|>{output}<|eot_id|>"
llama3_format_noprompt_train = "<|eot_id|><|start_header_id|>{input}<|end_header_id|>{output}<|eot_id|>"

# Common operations

## Clean memory

In [ ]:
gc.collect()
torch.cuda.empty_cache()

## Init entities

In [10]:
from transformers import GenerationConfig
from transformers.integrations import WandbCallback
import random
class LLMSampleCB(WandbCallback):
    def __init__(self, trainer, test_dataset, num_samples=2, max_new_tokens=2048, log_model="checkpoint"):
        super().__init__()
        self.sample_dataset = test_dataset.select(random.choices(range(len(test_dataset)), k=num_samples))
        self.model, self.tokenizer = trainer.model, trainer.tokenizer
        self.inference_model = None
        self.gen_config = GenerationConfig.from_pretrained(trainer.model.name_or_path,
                                                           max_new_tokens=max_new_tokens)
        self.gen_config.temperature = 0.9
    def generate(self, prompt):
        tokenized_prompt = self.tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
        with torch.inference_mode():
            output = self.inference_model.generate(**{"input_ids":tokenized_prompt,
                                                      "generation_config":self.gen_config})
        return self.tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)

    def samples_table(self, examples):
        records_table = wandb.Table(columns=["prompt", "generation"] + list(self.gen_config.to_dict().keys()))
        for example in tqdm(examples, leave=False):
            prompt = get_inference_text(example['natural_input'], tokenizer)
            generation = self.generate(prompt=prompt)
            records_table.add_data(prompt, generation, *list(self.gen_config.to_dict().values()))
        return records_table

    def on_evaluate(self, args, state, control,  **kwargs):
        super().on_evaluate(args, state, control, **kwargs)
        self.inference_model = self.model
        records_table = self.samples_table(self.sample_dataset)
        self._wandb.log({"sample_predictions":records_table})

In [19]:
def generate(model, tokenizer, generation_config, inp):
  tokenized_prompt = tokenizer(inp, return_tensors='pt', padding=True, truncation=True)
  tokenized_prompt_ids = tokenized_prompt["input_ids"].cuda()
  tokenized_prompt_mask = tokenized_prompt["attention_mask"].cuda()
  with torch.inference_mode():
      output = model.generate(**{"input_ids":tokenized_prompt_ids, "attention_mask":tokenized_prompt_mask, "generation_config":generation_config}).detach().cpu()
  decoded = []
  for i in range(output.shape[0]):
    ans = tokenizer.decode(output[i][len(tokenized_prompt[0]):], skip_special_tokens=True)
    decoded.append(ans)
  return decoded

In [12]:
def transform_to_nl(dataset, batch_size, text2text_model, tokenizer, gen_config, prompt, path):
  natural_inputs = {}
  if os.path.exists(path):
      with open(path, "r") as f:
        current = json.load(f)
        for key in current.keys():
            natural_inputs[key] = current[key]
  n_batches = int(len(dataset)/batch_size)
  if n_batches * batch_size < len(dataset):
      n_batches += 1
  for i in range(n_batches):
      rest_inds = [ind for ind in range(len(dataset)) if str(ind) not in natural_inputs]
      if len(rest_inds) == 0:
          continue
      if len(rest_inds) < batch_size:
          inds = rest_inds
      else:
          inds = random.sample(rest_inds, batch_size)
      batch = dataset.select(inds)
      prompts = [get_inference_text(prompt, inp, tokenizer) for inp in batch["input"]]
      generations = generate(text2text_model, tokenizer, gen_config, prompts)
      
      for ind, pos in enumerate(inds):
          natural_inputs[pos] = generations[ind]
      with open(path, "w+") as f:
        json.dump(natural_inputs, f)
      print(f"Batch {i} is processed")
  return natural_inputs

### Filtering dataset to delete poor samples

In [54]:
def clean_dataset(row):
    result = True
    row = {"input": row["input"].lower(), "natural_input": row["natural_input"].lower()}
    if "can't" in row["natural_input"] or "can not" in row["natural_input"]:
        result = False
    ints = ["length", "width", "heignt"]
    for param in ints:
        if param in row["input"]:
            value = re.findall("(?<=" + param + ": )\d+?(?= cm)", row["input"])
            if len(value) == 0:
                continue
            else:
                value = value[0]
            equals = [" "+value+" ", value[:-2] + " \w+? " + value[-2:], " "+str(float(value)/100)+" "]
            found = False
            for e in equals:
                if len(re.findall(e, row["natural_input"])) > 0:
                    found = True
                    break
            if not found:
                result = False
                break
    doubles = ["wheel radius", "wheel width"]
    for param in doubles:
        if param in row["input"]:
            value = re.findall("(?<=" + param + ": )\d+?(?= cm)", row["input"])
            if len(value) == 0:
                continue
            else:
                value = value[0]
            if len(re.findall(" "+value+" ", row["natural_input"])) == 0:
                result = False
                break
    body_types = ["sedan", "hatchback", "station wagon", "coupe", "limousine", "SUV", "pickup", "minivan", "van", "truck"]
    if "body type" in row["input"]:
        found = []
        for body in body_types:
            if body in row["natural_input"]:
                found.append(body)
        if len(found) > 1:
            result = False
        if len(found) == 1:
            right_body = re.findall("(?<=body type: )\w+?", row["input"])[0]
            if right_body != found[0]:
                result = False
    return result

### Text formatting with custom template (codellama)

In [ ]:
eos_token = "<|end_of_text|>"
b_inst_token = "<|start_header_id|>"
e_inst_token = "<|end_header_id|>"
def get_inference_text(prompt, text, tokenizer):
  input_format_without_prompt = """{b_inst}user{e_inst}
  {text}{eos}{b_inst}assistant{e_inst}
  """
  input_format_with_prompt = """{b_inst}system{e_inst}
  {prompt}{eos}""" + input_format_without_prompt
  input_text = ""
  if len(prompt) == 0:
    input_text = input_format_without_prompt.format(b_inst=b_inst_token, text=text, e_inst=e_inst_token, eos=eos_token)
  else:
    input_text = input_format_with_prompt.format(prompt=prompt, b_inst=b_inst_token, text=text, e_inst=e_inst_token, eos=eos_token)
  return input_text
def get_train_text(prompt, input_text, output_text, tokenizer):
  output_format = """{output}{eos}
  """
  inp = get_inference_text(prompt, input_text, tokenizer)
  out = inp + output_format.format(output=output_text, eos=eos_token)
  return out
def generate_train_text(prompt, row, tokenizer):
  row["text"] = get_train_text(prompt, row["natural_input"], row["output"])
  return row

### Text formatting with huggingface chat template (qwen, mistral)

In [13]:
def get_inference_text(prompt, text, tokenizer):
  messages = [
      {"role": "system", "content": prompt},
      {"role": "user", "content": text}
  ]
  input_text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  return input_text
def get_train_text(prompt, input_text, output_text, tokenizer):
  messages = [
      {"role": "system", "content": prompt},
      {"role": "user", "content": input_text},
      {"role": "assistant", "content": output_text},
  ]
  out = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
  )
  return out

def generate_train_text(prompt, row, tokenizer):
  row["text"] = get_train_text(prompt, row["natural_input"], row["output"], tokenizer)
  return row

## Huggingface login

In [25]:
login()

# Dataset creation

## Generate natural language descriptions

In [80]:
dataset = load_dataset(dataset_name)

In [8]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_compute_dtype=torch.float16,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_use_double_quant=True)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(text2text_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_params = LoraConfig(
    lora_alpha=16, lora_dropout=0.1, r=64, bias="none", task_type="CAUSAL_LM")

In [10]:
text2text_model = AutoModelForCausalLM.from_pretrained(text2text_model_name,
                                             quantization_config=quant_config,
                                             device_map='auto',
                                  low_cpu_mem_usage=True, offload_state_dict=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [81]:
train_dataloader = DataLoader(dataset["train"], batch_size=256, shuffle=False)
test_dataloader = DataLoader(dataset["test"], batch_size=256, shuffle=False)

In [12]:
gen_config = GenerationConfig.from_pretrained(text2text_model_name)
gen_config.max_length = 2048
prompt = """You receive a request to generate a car in the format "Generate a car with the following characteristics: name1:value1; name2:value2;...". This request contains a set of car characteristics. Rewrite this request into human language without losing information about the characteristics of the car. You must return a response in the following json format: {"rewritten request": here you must paste the rewritten request in human language, "explanation": here you should explain where in the human language query the information about a specific parameter is mentioned. Write it in the following json format: {parameter name: mention explanation} } Here are some examples:
Example 1
request: Generate a car with the following characteristics: length: 290 cm; width: 150 cm; height: 135 cm; wheel width: 27 cm; wheel radius: 36.6 cm; clearance height: 12 cm; bonnet size : small; bonnet shape : rounded; windscreen shape: rounded; boot shape: rounded;
answer: {"rewritten request":"Make me a car with a rounded hood and windshield, wheels with a radius of 36.6 cm and a width of 27 cm. The clearance should be 12 cm, the length 290 cm, the width 150 cm, the height 135 cm. Also let the hood be small and boot be rounded.",
"explanation":{"length": "I mentioned, that 'the length 290 cm'", "width":"I mentioned, that 'the width 150 cm'", "height": "I mentioned, that 'the height 135 cm'", "wheel width":"I mentioned it here: 'wheels with a radius of 36.6 cm and a width of 27 cm'", "wheel radius": "I mentioned it here: 'wheels with a radius of 36.6 cm and a width of 27 cm'", "clearance height": "I mentioned, that 'The clearance should be 12 cm'", "bonnet size": "I mentioned it here: 'let the hood be small'", "bonnet shape":"I mentioned it here: 'a car with a rounded hood', "windscreen shape": "I mentioned it here: 'a car with a rounded hood and windshield'", "boot shape": "I mentioned it here: 'let the hood be small and boot be rounded'"}}
Example 2
request: Generate a car with the following characteristics: length: 350 cm; width: 150 cm; height: 135 cm; wheel width: 15 cm; wheel radius: 35.15 cm; clearance height: 14 cm; bonnet size : large; bonnet shape : rounded; windscreen shape: rounded; boot shape: flat;
answer: {"rewritten request":"Generate a car with a length of 350 cm, a width of 150 cm and a height of 135 cm. Make a large rounded hood, a flat trunk and a rounded windshield. The wheels should have a radius of 35.15 cm and a width of 15 cm. The clearance should be 14 cm.",
"explanation":{"length": "I mentioned it here 'a car with a length of 350 cm'", "width":"I mentioned, that 'a width of 150 cm'", "height": "I mentioned, that 'a height of 135 cm'", "wheel width":"I mentioned it here: 'The wheels should have a radius of 35.15 cm and a width of 15 cm'", "wheel radius": "I mentioned it here: 'The wheels should have a radius of 35.15 cm and a width of 15 cm'", "clearance height": "I mentioned, that 'The clearance should be 14 cm'", "bonnet size": "I mentioned it here: 'Make a large rounded hood'", "bonnet shape":"I mentioned it here: 'Make a large rounded hood', "windscreen shape": "I mentioned it here: 'a rounded windshield'", "boot shape": "I mentioned it here: 'a flat trunk'"}}
Example 3
request: Generate a car with the following characteristics: 2 wheelsets;normal size boot; length: 250 cm; height: 135 cm; wheel radius: 25 cm;
answer: {"rewritten request":"Create a passenger car with a standard-sized trunk. It will be 2.5 meters long and 1 meter 35 centimeters high. The wheels should have a radius of 25 centimeters.",
"explanation":{"wheelsets count":"I mentioned it here: 'Create a passenger car'", "boot size":"I mentioned it here: 'standard-sized trunk'", "length":"I mentioned it here: 'It will be 2.5 meters long'", "height": "I mentioned it here: '1 meter 35 centimeters high'", "wheel radius": "I mentioned it here: 'The wheels should have a radius of 25 centimeters.'"}}
rewritten request: Create a passenger car with a standard-sized trunk. It will be 2.5 meters long and 1 meter 35 centimeters high. The wheels should have a radius of 25 centimeters.
Example 4
request: Generate a car with the following characteristics: body type: truck;
answer: {"rewritten request": "I want a random truck",
"explanation": {"body type":"I mentioned it here: 'a random truck'}}
Example 5
request: Generate a car with the following characteristics: normal size bonnet; length: 250 cm; width is small; height is small; wheel radius: 25 cm;
answer: {"rewritten request":"I want you to create a car with a wheel radius of 25 cm. Make it compact, but make it 2.5 meters long. Leave the hood of standard size.",
"explanation":{"bonnet size":"I mentioned it here: 'Leave the hood of standard size', "length": "I mentioned it here: 'make it 2.5 meters long'", "width":"I mentioned it here: 'Make it compact'", "height":"I mentioned it here: 'Make it compact'", "wheel radius": "I mentioned it here: 'a wheel radius of 25 cm'"}}
"""

In [ ]:
gen_config = GenerationConfig.from_pretrained(text2text_model_name)
gen_config.max_length = 2048
prompt = """You receive a request to generate a car in the format "Generate a car with the following characteristics: name1:value1; name2:value2;...". This request contains a set of car characteristics. Rewrite this request into human language without losing information about the characteristics of the car. You must return a response in the following json format: {"rewritten request": here you must paste the rewritten request in human language, "explanation": here you should explain where in the human language query the information about a specific parameter is mentioned } Here are some examples:
Example 1
request: Generate a car with the following characteristics: length: 290 cm; width: 150 cm; height: 135 cm; wheel width: 27 cm; wheel radius: 36.6 cm; clearance height: 12 cm; bonnet size : small; bonnet shape : rounded; windscreen shape: rounded; boot shape: rounded;
answer: {"rewritten request":"Make me a car with a rounded hood and windshield, wheels with a radius of 36.6 cm and a width of 27 cm. The clearance should be 12 cm, the length 290 cm, the width 150 cm, the height 135 cm. Also let the hood be small and boot be rounded.",
"explanation":"I mentioned the clearance height, length, width and height here: 'clearance should be 12 cm, the length 290 cm, the width 150 cm, the height 135 cm'. I wrote about the wheel parameters here: 'wheels with a radius of 36.6 cm and a width of 27 cm'. Here I wrote about the hood: 'let the hood be small', 'a car with a rounded hood', about the windscreen: 'a rounded hood and windshield', and about the boot: 'boot be rounded'."}
Example 2
request: Generate a car with the following characteristics: length: 350 cm; width: 150 cm; height: 135 cm; wheel width: 15 cm; wheel radius: 35.15 cm; clearance height: 14 cm; bonnet size : large; bonnet shape : rounded; windscreen shape: rounded; boot shape: flat;
answer: {"rewritten request":"Generate a car with a length of 350 cm, a width of 150 cm and a height of 135 cm. Make a large rounded hood, a flat trunk and a rounded windshield. The wheels should have a radius of 35.15 cm and a width of 15 cm. The clearance should be 14 cm.",
"explanation":"I mentioned the length, width and height here: 'a length of 350 cm, a width of 150 cm and a height of 135 cm'. Wheel parameters have been mentioned here: 'The wheels should have a radius of 35.15 cm and a width of 15 cm'. Here I wrote about the clearance height: 'The clearance should be 14 cm', and here I wrote about the body shapes: 'Make a large rounded hood, a flat trunk and a rounded windshield'."}
Example 3
request: Generate a car with the following characteristics: 2 wheelsets;normal size boot; length: 250 cm; height: 135 cm; wheel radius: 25 cm;
answer: {"rewritten request":"Create a passenger car with a standard-sized trunk. It will be 2.5 meters long and 1 meter 35 centimeters high. The wheels should have a radius of 25 centimeters.",
"explanation":"I wrote 'Create a passenger car', because every passenger car has 2 wheelsets. 'standard-sized trunk' means normal size boot. Length and height are mentioned here: 'It will be 2.5 meters long and 1 meter 35 centimeters high'. And here I wrote about wheel radius: 'The wheels should have a radius of 25 centimeters'."}
Example 4
request: Generate a car with the following characteristics: body type: truck;
answer: {"rewritten request": "I want a random truck",
"explanation": "The request only mentions the body type, thats why I wrote 'random truck'."}
Example 5
request: Generate a car with the following characteristics: normal size bonnet; length: 250 cm; width is small; height is small; wheel radius: 25 cm;
answer: {"rewritten request":"I want you to create a car with a wheel radius of 25 cm. Make it compact, but make it 2.5 meters long. Leave the hood of standard size.",
"explanation":"I wrote 'Make it compact', because both width and height are small. I wrote about length here: 'make it 2.5 meters long'. Here I mentioned the wheel radius: 'wheel radius of 25 cm'. And here I wrote about the bonnet size: 'Leave the hood of standard size'."}
"""

In [16]:
train_nl = transform_to_nl(train_dataloader, 0, text2text_model, tokenizer, gen_config, prompt, "train_nl.json")

  0%|          | 0/10 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [28]:
dataset["train"] = dataset["train"].add_column("natural_input_mistral", train_nl)

In [29]:
def fix_nl(row):
    return row

In [30]:
dataset["train"] = dataset["train"].map(fix_nl)

In [13]:
test_nl = transform_to_nl(test_dataloader, 0, text2text_model, tokenizer, gen_config, prompt, "test_nl.txt")

  0%|          | 0/101 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pa

In [15]:
dataset["test"] = dataset["test"].add_column("natural_input_mistral", test_nl)

In [18]:
dataset["test"] = dataset["test"].map(fix_nl)

Map:   0%|          | 0/25769 [00:00<?, ? examples/s]

In [34]:
dataset.push_to_hub(dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/54 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/evgmaslov/cars/commit/69597e7885381c5415d966515efdecaa5e6dcff5', commit_message='Upload dataset', commit_description='', oid='69597e7885381c5415d966515efdecaa5e6dcff5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/evgmaslov/cars', endpoint='https://huggingface.co', repo_type='dataset', repo_id='evgmaslov/cars'), pr_revision=None, pr_num=None)

## Validate dataset

In [55]:
dataset = load_dataset(dataset_name)

In [56]:
poor_samples = []

In [60]:
print(dataset["train"][0]["natural_input"])
print(dataset["train"][0]["input"])

Make me a car that is compact and has a small trunk. The clearance should be low, the length should be short, the width 167 cm and the height should be low.
Generate a car with the following characteristics: normal size boot; length is small; width: 167 cm; height is small; wheel radius is small


# Training

## Prepare dataset

In [14]:
prompt = """You are an experienced engineer who designs cars. You know everything about the shape and size of cars. You know many different types of car bodies and understand their geometric characteristics. You help people get the car they need. You receive a request to create a car, which describes its shape, size, and other characteristics. To create a car, you use a C# class of the following structure:
public Car()
{
    Length = ...;
    Width = ...;
    Height = ...;

    WheelWidth = ...;
    WheelRadius = ...;

    WheelRelativeBiasAlongWidth = ...;
    WheelRelativeBiasesAlongLength = ...;

    WheelBaseSegmentsSpans = ...;
    WheelBaseSegmentsBottomSurfaces = ...;
    WheelBaseTopSurface = ...;
    GapBetweenWheelAndBase = ...;

    BodySegmentsSpans = ...;
    BodySegmentsTopSurfaces = ...;
}.
Here is a description of its parameters. "Length" is the length of the car in centimeters, this is a float type parameter. "Width" is the width of the car in centimeters, this is a float type parameter. "Height" is the width of the car in centimeters, this is a float type parameter. "WheelWidth" is the width of each wheel in centimeters, this is a float type parameter. "WheelRadius" is the radius of each wheel in centimeters, this is a float type parameter. "WheelRelativeBiasAlongWidth" is a parameter that determines how deep the wheel is shifted into the car body. For example, if WheelRelativeBiasAlongWidth = 0.1, then the wheel is shifted into the car by a distance equal to 10% of its width. This parameter is of float type. "WheelRelativeBiasesAlongLength" is a parameter that contains a list of numbers in floating point format. The number of values in the list is equal to the number of wheel pairs the car has. Each value is equal to the relative offset of a wheel pair relative to the front of the car. For example, if WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f }, then the car has two wheel pairs, one offset by 20% of the length, and the other by 80%. The "WheelBaseSegmentsSpans" parameter describes the arrangement of the parts of the car's floor. The floor of a car is the frame on which its wheels are mounted. Usually a car has one frame, but a truck can have 2 or more frames. This parameter contains a list where each element is a range of lengths in which the floor of the car is located. The range is described by a list where the first element is the start of the range and the second element is the end of the range. The WheelBaseSegmentsBottomSurfaces parameter describes the shape of the bottom of the car. If the bottom of the car consists of several planes, this parameter will contain several planes. This parameter contains as many planes as there are ranges specified in the "WheelBaseSegmentsSpans" parameter, because each plane has its own range. This parameter can be used to adjust the ground clearance of the car. The "BodySegmentsSpans" parameter contains a list of length ranges in which the vehicle body components are located. The "BodySegmentsTopSurfaces" parameter contains a list of the components of the car body. The number of components of the car is equal to the number of ranges in the "BodySegmentsSpans" parameter, because each component has its own range. Each component of the car is described by the shape of its surface. The shape can be specified by one of three classes: Constant (plain shape), CornerRounded (plain shape with rounded corners) and TotalRounded (rounded shape).
The user will ask you to generate a car and will provide you with a description of its shape and dimensions. You need to extract from his request the information needed to fill in the C# class parameters described above. Write C# class to generate a car, fill in its parameters according to the user's request. If the user has not specified specific dimensions or has provided insufficient information, use all your knowledge of car shapes and sizes to fill in the missing values and generate the correct car class. Here are some examples of what you need to do:
Example 1
user request: Design a car with a standard-sized hood and trunk. It should be 1.67 meters wide and have wheels with a radius of 40 cm.
car: using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 150f;

            WheelWidth = 20f;
            WheelRadius = 40f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 13f) };
            WheelBaseTopSurface = new Constant(height: 90f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.3f }, new List<float>(){ 0.3f, 0.8f }, new List<float>(){ 0.8f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new Constant(height: 105f), new Constant(height: 150f), new CornerRounded(minHeight: 90f, maxHeight: 105f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 49.999996f, leftCornerRounded: false, rightCornerRounded: true) };
        }
    }

}
Example 2:
user request: Design a car with two wheelsets and a standard-sized hood. It should be 1.67 meters wide.
car: using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 150f;

            WheelWidth = 20f;
            WheelRadius = 25f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 18f) };
            WheelBaseTopSurface = new Constant(height: 75f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.3f }, new List<float>(){ 0.3f, 0.9f }, new List<float>(){ 0.9f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new TotalRounded(minHeight: 75f, maxHeight: 90f, leftRounded: true), new Constant(height: 150f), new CornerRounded(minHeight: 75f, maxHeight: 90f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 25.000006f, leftCornerRounded: false, rightCornerRounded: true) };
        }
    }

}
Example 3: 
user request: Design a compact sedan. It should be short and narrow, with a small height.
car: using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 135f;

            WheelWidth = 20f;
            WheelRadius = 25f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 13f) };
            WheelBaseTopSurface = new Constant(height: 67.5f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.2f }, new List<float>(){ 0.2f, 0.8f }, new List<float>(){ 0.8f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new CornerRounded(minHeight: 67.5f, maxHeight: 94.5f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 50f, leftCornerRounded: true, rightCornerRounded: false), new CornerRounded(minHeight: 94.5f, maxHeight: 135f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 150f, leftCornerRounded: true, rightCornerRounded: true), new CornerRounded(minHeight: 67.5f, maxHeight: 94.5f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 49.999996f, leftCornerRounded: false, rightCornerRounded: true) };
        }
    }

}
Example 4:
user request: Design a car with two wheelsets, a standard-sized hood, and a height of 1.35 meters. Ensure the wheel radius is large.
car: using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 135f;

            WheelWidth = 20f;
            WheelRadius = 40f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 18f) };
            WheelBaseTopSurface = new Constant(height: 94.5f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.2f }, new List<float>(){ 0.2f, 0.8f }, new List<float>(){ 0.8f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new TotalRounded(minHeight: 94.5f, maxHeight: 108f, leftRounded: true), new CornerRounded(minHeight: 108f, maxHeight: 135f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 150f, leftCornerRounded: true, rightCornerRounded: true), new TotalRounded(minHeight: 94.5f, maxHeight: 108f, leftRounded: false) };
        }
    }

}"""

In [139]:
tokenizer = AutoTokenizer.from_pretrained(text2code_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

For mistral-nemo fix bug in chat template

In [148]:
tokenizer.chat_template = """{%- if messages[0][\"role\"] == \"system\" %}\n    {%- set system_message = messages[0][\"content\"] %}\n    {%- set loop_messages = messages[1:] %}\n{%- else %}\n    {%- set loop_messages = messages %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n{%- set user_messages = loop_messages | selectattr(\"role\", \"equalto\", \"user\") | list %}\n\n{#- This block checks for alternating user/assistant messages, skipping tool calling messages #}\n{%- set ns = namespace() %}\n{%- set ns.index = 0 %}\n{%- for message in loop_messages %}\n    {%- if not (message.role == \"tool\" or message.role == \"tool_results\" or (message.tool_calls is defined and message.tool_calls is not none)) %}\n        {%- if (message[\"role\"] == \"user\") != (ns.index % 2 == 0) %}\n            {{- raise_exception(\"After the optional system message, conversation roles must alternate user/assistant/user/assistant/...\") }}\n        {%- endif %}\n        {%- set ns.index = ns.index + 1 %}\n    {%- endif %}\n{%- endfor %}\n\n{{- bos_token }}\n{%- for message in loop_messages %}\n    {%- if message[\"role\"] == \"user\" %}\n        {%- if tools is not none and (message == user_messages[-1]) %}\n            {{- \"[AVAILABLE_TOOLS][\" }}\n            {%- for tool in tools %}\n                {%- set tool = tool.function %}\n                {{- '{\"type\": \"function\", \"function\": {' }}\n                {%- for key, val in tool.items() if key != \"return\" %}\n                    {%- if val is string %}\n                        {{- '\"' + key + '\": \"' + val + '\"' }}\n                    {%- else %}\n                        {{- '\"' + key + '\": ' + val|tojson }}\n                    {%- endif %}\n                    {%- if not loop.last %}\n                        {{- \", \" }}\n                    {%- endif %}\n                {%- endfor %}\n                {{- \"}}\" }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- else %}\n                    {{- \"]\" }}\n                {%- endif %}\n            {%- endfor %}\n            {{- \"[/AVAILABLE_TOOLS]\" }}\n            {%- endif %}\n        {%- if system_message is defined %}\n            {{- \"[INST]\" + system_message + \"\\n\\n\" + message[\"content\"] + \"[/INST]\" }}\n        {%- else %}\n            {{- \"[INST]\" + message[\"content\"] + \"[/INST]\" }}\n        {%- endif %}\n    {%- elif (message.tool_calls is defined and message.tool_calls is not none) %}\n        {{- \"[TOOL_CALLS][\" }}\n        {%- for tool_call in message.tool_calls %}\n            {%- set out = tool_call.function|tojson %}\n            {{- out[:-1] }}\n            {%- if not tool_call.id is defined or tool_call.id|length != 9 %}\n                {{- raise_exception(\"Tool call IDs should be alphanumeric strings with length 9!\") }}\n            {%- endif %}\n            {{- ', \"id\": \"' + tool_call.id + '\"}' }}\n            {%- if not loop.last %}\n                {{- \", \" }}\n            {%- else %}\n                {{- \"]\" + eos_token }}\n            {%- endif %}\n        {%- endfor %}\n    {%- elif message[\"role\"] == \"assistant\" %}\n        {{- message[\"content\"] + eos_token}}\n    {%- elif message[\"role\"] == \"tool_results\" or message[\"role\"] == \"tool\" %}\n        {%- if message.content is defined and message.content.content is defined %}\n            {%- set content = message.content.content %}\n        {%- else %}\n            {%- set content = message.content %}\n        {%- endif %}\n        {{- '[TOOL_RESULTS]{\"content\": ' + content|string + \", \" }}\n        {%- if not message.tool_call_id is defined or message.tool_call_id|length != 9 %}\n            {{- raise_exception(\"Tool call IDs should be alphanumeric strings with length 9!\") }}\n        {%- endif %}\n        {{- '\"call_id\": \"' + message.tool_call_id + '\"}[/TOOL_RESULTS]' }}\n    {%- else %}\n        {{- raise_exception(\"Only user and assistant roles are supported, with the exception of an initial optional system message!\") }}\n    {%- endif %}\n{%- endfor %}\n"""

In [18]:
dataset = dataset.filter(clean_dataset)

In [19]:
dataset = dataset.map(lambda row: generate_train_text(row, tokenizer))

Map:   0%|          | 0/33866 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

## Init model

In [121]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_compute_dtype=torch.float16,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_use_double_quant=True)
peft_params = LoraConfig(
    lora_alpha=16, lora_dropout=0.1, r=64, bias="none", task_type="CAUSAL_LM")

In [95]:
model = AutoModelForCausalLM.from_pretrained(text2code_model_name,
                                             quantization_config=quant_config,
                                             device_map='auto',
                                  low_cpu_mem_usage=True, offload_state_dict=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Train

In [33]:
os.environ["WANDB_PROJECT"]="leap71"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [34]:
resume_from_checkpoint=False
training_params = TrainingArguments(
    output_dir=text2code_tuned_model_name,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=100, 
    learning_rate=3e-5, weight_decay=0.001,
    fp16=False, bf16=False,
    max_grad_norm=0.3,
    max_steps=-1, warmup_ratio=0.01,
    group_by_length=True, lr_scheduler_type='cosine_with_restarts',
    lr_scheduler_kwargs={"num_cycles":5},
    report_to="wandb",
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=100,
    push_to_hub=True,
    hub_strategy="checkpoint",
    resume_from_checkpoint=resume_from_checkpoint,
    run_name="natural_language_to_code_1")
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"].select(range(32)),
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params, packing=False)
wandb_callback = LLMSampleCB(trainer, dataset["test"], num_samples=2, max_new_tokens=2048)
trainer.add_callback(wandb_callback)

/home/jovyan/.mlspace/envs/leap71/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/jovyan/.mlspace/envs/leap71/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/jovyan/.mlspace/envs/leap71/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/53280 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [35]:
!wandb login

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: free001style. Use `wandb login --relogin` to force relogin


In [21]:
wandb.init("leap71")
if resume_from_checkpoint:
  #trainer.train(resume_from_checkpoint=text2code_tuned_model_name + "/last-checkpoint")
  trainer.train(resume_from_checkpoint=text2code_tuned_model_name + "/checkpoint-10000")
else:
  trainer.train()

/home/jovyan/.mlspace/envs/leap71/lib/python3.10/site-packages/transformers/trainer.py:3262: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpoint

Step,Training Loss,Validation Loss
10500,0.049800,0.316373
11000,0.050100,0.298745
11500,0.049700,0.314826
12000,0.048800,0.310907
12500,0.048600,0.290600
13000,0.047500,0.296261
13500,0.047800,0.295860
14000,0.047300,0.291479
14500,0.047600,0.285518
15000,0.046500,0.292918


  0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
wandb: Adding directory to artifact (./evgmaslov/Code-Llama-3-8B-cars/checkpoint-10500)... Done. 1.4s
wandb: Adding directory to artif

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
wandb: Adding directory to artifact (./evgmaslov/Code-Llama-3-8B-cars/checkpoint-14500)... Done. 1.8s
wandb: Adding directory to artifact (./evgmaslov/Code-Llama-3-8B-cars/checkpoint-14500)... Done. 1.4s


  0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
wandb: Adding directory to artifact (./evgmaslov/Code-Llama-3-8B-cars/checkpoint-15000)... Done. 1.4s
wandb: Adding directory to artifact (./evgmaslov/Code-Llama-3-8B-cars/checkpoint-15000)... Done. 1.3s


KeyboardInterrupt: 

# Inference

In [20]:
tokenizer = AutoTokenizer.from_pretrained(text2code_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.chat_template = """{%- if messages[0][\"role\"] == \"system\" %}\n    {%- set system_message = messages[0][\"content\"] %}\n    {%- set loop_messages = messages[1:] %}\n{%- else %}\n    {%- set loop_messages = messages %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n{%- set user_messages = loop_messages | selectattr(\"role\", \"equalto\", \"user\") | list %}\n\n{#- This block checks for alternating user/assistant messages, skipping tool calling messages #}\n{%- set ns = namespace() %}\n{%- set ns.index = 0 %}\n{%- for message in loop_messages %}\n    {%- if not (message.role == \"tool\" or message.role == \"tool_results\" or (message.tool_calls is defined and message.tool_calls is not none)) %}\n        {%- if (message[\"role\"] == \"user\") != (ns.index % 2 == 0) %}\n            {{- raise_exception(\"After the optional system message, conversation roles must alternate user/assistant/user/assistant/...\") }}\n        {%- endif %}\n        {%- set ns.index = ns.index + 1 %}\n    {%- endif %}\n{%- endfor %}\n\n{{- bos_token }}\n{%- for message in loop_messages %}\n    {%- if message[\"role\"] == \"user\" %}\n        {%- if tools is not none and (message == user_messages[-1]) %}\n            {{- \"[AVAILABLE_TOOLS][\" }}\n            {%- for tool in tools %}\n                {%- set tool = tool.function %}\n                {{- '{\"type\": \"function\", \"function\": {' }}\n                {%- for key, val in tool.items() if key != \"return\" %}\n                    {%- if val is string %}\n                        {{- '\"' + key + '\": \"' + val + '\"' }}\n                    {%- else %}\n                        {{- '\"' + key + '\": ' + val|tojson }}\n                    {%- endif %}\n                    {%- if not loop.last %}\n                        {{- \", \" }}\n                    {%- endif %}\n                {%- endfor %}\n                {{- \"}}\" }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- else %}\n                    {{- \"]\" }}\n                {%- endif %}\n            {%- endfor %}\n            {{- \"[/AVAILABLE_TOOLS]\" }}\n            {%- endif %}\n        {%- if system_message is defined %}\n            {{- \"[INST]\" + system_message + \"\\n\\n\" + message[\"content\"] + \"[/INST]\" }}\n        {%- else %}\n            {{- \"[INST]\" + message[\"content\"] + \"[/INST]\" }}\n        {%- endif %}\n    {%- elif (message.tool_calls is defined and message.tool_calls is not none) %}\n        {{- \"[TOOL_CALLS][\" }}\n        {%- for tool_call in message.tool_calls %}\n            {%- set out = tool_call.function|tojson %}\n            {{- out[:-1] }}\n            {%- if not tool_call.id is defined or tool_call.id|length != 9 %}\n                {{- raise_exception(\"Tool call IDs should be alphanumeric strings with length 9!\") }}\n            {%- endif %}\n            {{- ', \"id\": \"' + tool_call.id + '\"}' }}\n            {%- if not loop.last %}\n                {{- \", \" }}\n            {%- else %}\n                {{- \"]\" + eos_token }}\n            {%- endif %}\n        {%- endfor %}\n    {%- elif message[\"role\"] == \"assistant\" %}\n        {{- message[\"content\"] + eos_token}}\n    {%- elif message[\"role\"] == \"tool_results\" or message[\"role\"] == \"tool\" %}\n        {%- if message.content is defined and message.content.content is defined %}\n            {%- set content = message.content.content %}\n        {%- else %}\n            {%- set content = message.content %}\n        {%- endif %}\n        {{- '[TOOL_RESULTS]{\"content\": ' + content|string + \", \" }}\n        {%- if not message.tool_call_id is defined or message.tool_call_id|length != 9 %}\n            {{- raise_exception(\"Tool call IDs should be alphanumeric strings with length 9!\") }}\n        {%- endif %}\n        {{- '\"call_id\": \"' + message.tool_call_id + '\"}[/TOOL_RESULTS]' }}\n    {%- else %}\n        {{- raise_exception(\"Only user and assistant roles are supported, with the exception of an initial optional system message!\") }}\n    {%- endif %}\n{%- endfor %}\n"""

In [21]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_compute_dtype=torch.float16,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_use_double_quant=True)
peft_params = LoraConfig(
    lora_alpha=16, lora_dropout=0.1, r=64, bias="none", task_type="CAUSAL_LM")

In [22]:
model = AutoModelForCausalLM.from_pretrained(text2code_model_name,
                                             quantization_config=quant_config,
                                             device_map='auto',
                                  low_cpu_mem_usage=True, offload_state_dict=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [23]:
model = PeftModel.from_pretrained(model, "evgmaslov/exp7-Mistral-Nemo-Instruct-2407-cars")

In [27]:
model.push_to_hub(text2code_tuned_model_name)

CommitInfo(commit_url='https://huggingface.co/evgmaslov/Mistral-Nemo-Instruct-2407-cars/commit/bc542a155f0691ec8330875bfe649cbbac8b2af7', commit_message='Upload model', commit_description='', oid='bc542a155f0691ec8330875bfe649cbbac8b2af7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/evgmaslov/Mistral-Nemo-Instruct-2407-cars', endpoint='https://huggingface.co', repo_type='model', repo_id='evgmaslov/Mistral-Nemo-Instruct-2407-cars'), pr_revision=None, pr_num=None)

In [274]:
user_prompt = "Craft a hatchback with the length of 3.5 metres and the door at the rear."
gen_config = GenerationConfig.from_pretrained(text2code_model_name)
gen_config.max_length = 4096
output = generate(model, tokenizer, gen_config, get_inference_text(prompt, user_prompt, tokenizer))
print(output[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Here's a C# class representing a hatchback car with the specified dimensions and characteristics. I've justified each parameter choice below the class.

```csharp
using System.Collections.Generic;
using System;
using System.Collections;

namespace GenerativeDesign.Cars
{
    public class Hatchback : CarBase
    {
        public Hatchback()
        {
            Length = 350f;
            Width = 167f;
            Height = 165f;

            WheelWidth = 20f;
            WheelRadius = 30f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0f, 0.6f, 1f };

            WheelBaseSegmentsSpans = new List<List<float>>()
            {
                new List<float>() { 0f, 0.2f },
                new List<float>() { 0.2f, 0.8f },
                new List<float>() { 0.8f, 1f }
            };
            WheelBaseSegmentsBottomSurfaces = new List<Line>()
            {
                new Constant(height: 15f),
                new C

## Inference on the test set

In [6]:
dataset = load_dataset(dataset_name)

with open("train_nl.json", "r") as f:
    data = json.load(f)
small_dataset = {"input":[], "natural_input":[], "output":[], "text": []}
keys = [int(k) for k in data.keys()]
sorted_keys = sorted(keys)
for ind in tqdm(sorted_keys):
    natural_input = ""
    try:
        natural_input = json.loads(data[str(ind)])["rewritten request"]
    except:
        continue
    if natural_input != "":
        row = dataset["train"][ind]
        small_dataset["input"].append(row["input"])
        small_dataset["output"].append(row["output"])
        small_dataset["text"].append("")
        small_dataset["natural_input"].append(natural_input)
small_dataset = Dataset.from_dict(small_dataset)
print(f"Dataset length: {len(small_dataset)}")
small_dataset = small_dataset.train_test_split(test_size=0.05, seed=42)

  0%|          | 0/13462 [00:00<?, ?it/s]

Dataset length: 13442


In [7]:
small_dataset["test"]["natural_input"][232]

'Craft a compact, two-wheeled vehicle. It should measure 1.67 meters in width and 1.35 meters in height, with small-radius wheels.'

In [19]:
gen_config = GenerationConfig.from_pretrained(text2code_model_name)
gen_config.max_length = 4096

In [20]:
def inference_on_test(dataset, batch_size, text2text_model, tokenizer, gen_config, prompt, path):
  natural_inputs = {}
  if os.path.exists(path):
      with open(path, "r") as f:
        current = json.load(f)
        for key in current.keys():
            natural_inputs[key] = current[key]
  n_batches = int(len(dataset)/batch_size)
  if n_batches * batch_size < len(dataset):
      n_batches += 1
  for i in range(n_batches):
      rest_inds = [ind for ind in range(len(dataset)) if str(ind) not in natural_inputs]
      if len(rest_inds) == 0:
          continue
      if len(rest_inds) < batch_size:
          inds = rest_inds
      else:
          inds = random.sample(rest_inds, batch_size)
      batch = dataset.select(inds)
      prompts = [get_inference_text(prompt, inp, tokenizer) for inp in batch["input"]]
      generations = generate(text2text_model, tokenizer, gen_config, prompts)
      
      for ind, pos in enumerate(inds):
          natural_inputs[pos] = generations[ind]
      with open(path, "w+") as f:
        json.dump(natural_inputs, f)
      print(f"Batch {i} is processed")
  return natural_inputs

In [21]:
generations = inference_on_test(small_dataset["test"], 24, model, tokenizer, gen_config, prompt, "generations.json")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


KeyboardInterrupt: 

In [43]:
ind = 120
print(small_dataset["train"]["natural_input"][ind])
print(small_dataset["train"]["output"][ind])

Design a colossal truck with seven sets of wheels, featuring a roof fairing and a sleeper cab. This behemoth should measure 20 meters in length, 2.6 meters in width, and tower at 3.2 meters in height, with each wheel boasting a radius of 45 centimeters.
using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 2000f;
            Width = 260f;
            Height = 320f;

            WheelWidth = 20f;
            WheelRadius = 45f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.05f, 0.36f, 0.41f, 0.65f, 0.7f, 0.82f, 0.95f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.46f }, new List<float>(){ 0.55f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 30f), new Constant(height: 70f) };
            W

In [23]:
print(small_dataset["train"]["input"][ind])

Generate a car with the following characteristics: 7 wheelsets;roof fairing with sleeper cab; length: 2000 cm; width: 260 cm; height: 320 cm; wheel radius: 45 cm


In [49]:
inp = small_dataset["train"]["input"][ind]

In [50]:
row = dataset["train"].filter(lambda sample: sample["input"] == inp)[0]

Filter:   0%|          | 0/53280 [00:00<?, ? examples/s]

# Evaluation

## Backend functions

In [144]:
pattern = "(?<=WheelBaseSegmentsSpans)\s*?=\s*?new\s*?List\s*?<\s*?List\s*?<\s*?float\s*?>\s*?>\s*?\(\s*?\)\s*?\{(.*?)\}\s*?;"
text = "WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>() { 0f, 1f }, new List<float>() { 0f, 1f } };"
re.findall(pattern, text)

[' new List<float>() { 0f, 1f }, new List<float>() { 0f, 1f } ']

In [75]:
def code_to_json(code):
    code = re.sub("//(?:.|\s)+?(?=\n)", "", code)
    float_parameters = ["Length", "Width", "Height", "WheelWidth", "WheelRadius", "WheelRelativeBiasAlongWidth", "GapBetweenWheelAndBase"]
    serialized = {}
    for param in float_parameters:
        float_pattern = "(?<=" + param + ")\s*?=\s*?[\d\.]+?(?=f)"
        value = re.findall(float_pattern, code)[0]
        serialized[param] = float(value.replace("=", "").strip())

    param = "WheelRelativeBiasesAlongLength"
    pattern = "(?<=" + param + ")(?:.|\n)+?(?=;)"
    value_list = re.findall(pattern, code)[0]
    value_list = re.findall("(?<=\{)(?:.|\n)+?(?=\})", value_list)[0].split(",")
    values = []
    for value in value_list:
        value = re.findall(".+?(?=f)", value)[0].strip()
        values.append(float(value))
    serialized[param] = values

    params = ["WheelBaseSegmentsSpans", "BodySegmentsSpans"]
    for param in params:
        pattern = "(?<=" + param + ")\s*?=\s*?new\s+?List\s*?<\s*?List\s*?<\s*?float\s*?>\s*?>\s*?\(\s*?\)\s*?\{((?:.|\n)+?)\}\s*?;"
        list_of_lists = re.findall(pattern, code)[0]
        pattern = "(?<=\{)(?:.|\n)+?(?=\})"
        float_lists = re.findall(pattern, list_of_lists)
        total_lists = []
        for l in float_lists:
            floats = l.split(",")
            float_values = []
            for f in floats:
                value = re.findall(".+?(?=f)", f)[0].strip()
                float_values.append(float(value))
            total_lists.append(float_values)
        serialized[param] = total_lists

    params = ["WheelBaseSegmentsBottomSurfaces", "BodySegmentsTopSurfaces"]
    for param in params:
        pattern = "(?<=" + param + ")(?:.|\n)+?(?=;)"
        classes = re.findall(pattern, code)[0]
        classes = re.findall("(?<=\{)(?:.|\n)+?(?=\})", classes)[0]
        classes = re.findall("new\s+?\w+?\s*?\((?:.|\n)+?\)", classes)
        class_jsons = []
        for c in classes:
            class_json = []
            pattern = "(?<=new).+?(?=\()"
            class_name = re.findall(pattern, c)[0].strip()
            class_json.append(class_name)
            pattern = "(?<=\()(?:.|\n)+?(?=\))"
            parameters = re.findall(pattern, c)[0].split(",")
            parameters_json = {}
            for par in parameters:
                par = par.split(":")
                param_name = par[0].strip()
                param_value = re.findall("(?:\d|\.)+?(?=f)|\w+", par[1])[0].strip()
                try:
                    parameters_json[param_name] = float(param_value)
                except:
                    parameters_json[param_name] = param_value
            class_json.append(parameters_json)
            class_jsons.append(class_json)
        serialized[param] = class_jsons

    param = "WheelBaseTopSurface"
    class_json = []
    pattern = "(?<=" + param + ").+?(?=;)"
    class_value = re.findall(pattern, code)[0]
    pattern = "(?<=new).+?(?=\()"
    class_name = re.findall(pattern, class_value)[0].strip()
    class_json.append(class_name)
    pattern = "(?<=\().+?(?=\))"
    parameters = re.findall(pattern, class_value)[0].split(",")
    parameters_json = {}
    for par in parameters:
        par = par.split(":")
        param_name = par[0].strip()
        param_value = re.findall(".+?(?=f)|\s*?\w+?\s*?", par[1])[0].strip()
        try:
            parameters_json[param_name] = float(param_value)
        except:
            parameters_json[param_name] = param_value
    class_json.append(parameters_json)
    serialized[param] = class_json
    return serialized

In [256]:
with open("Code_0.cs", "r") as f:
    code = f.read()
test_json = code_to_json(code)

In [257]:
test_json

{'Length': '250',
 'Width': '167',
 'Height': '135',
 'WheelWidth': '20',
 'WheelRadius': '25',
 'WheelRelativeBiasAlongWidth': '0.1',
 'GapBetweenWheelAndBase': '3',
 'WheelRelativeBiasesAlongLength': ['0.2', '0.8'],
 'WheelBaseSegmentsSpans': [['0', '1']],
 'BodySegmentsSpans': [['0', '1']],
 'WheelBaseSegmentsBottomSurfaces': [['Constant', {'height': '13'}]],
 'BodySegmentsTopSurfaces': [['Constant', {'height': '81'}],
  ['Constant', {'height': '135'}],
  ['TotalRounded',
   {'minHeight': '67.5', 'maxHeight': '81', 'leftRounded': ''}]],
 'WheelBaseTopSurface': ['Constant', {'height': '67.5'}]}

In [8]:
def extract_codes(generations):
    codes = []
    pattern = "(?<=```csharp)(?:.|\s)+?(?=```)"
    for gen in tqdm(generations):
        code = re.findall(pattern, gen)
        if len(code) != 0:
            code = code[0]
            codes.append(code)
    return codes

In [9]:
def transform_codes_to_jsons(codes):
    inds = []
    jsons = []
    for ind, code in tqdm(enumerate(codes), total=len(codes)):
        try:
            code_json = code_to_json(code)
            jsons.append(code_json)
            inds.append(ind)
        except:
            continue
    return inds, jsons

### Backend functions for reconstruction score

In [158]:
def get_nearest_value(value, values_to_search):
    distance = 1e9
    nearest_value = 0
    for v in values_to_search:
        local_distance = abs(v - value)
        if local_distance < distance:
            distance = local_distance
            nearest_value = v
    return nearest_value

In [11]:
def get_parameter_value_part(code_json, indexes):
    value = code_json
    for ind in indexes:
        if type(value) == list:
            value = value[int(ind)]
        else:
            value = value[ind]
    return value

In [12]:
def get_indexes(value):
    indexes = []
    if type(value) == list:
        for i in range(len(value)):
            new_index = [i]
            local_value = value[i]
            local_indexes = get_indexes(local_value)
            if len(local_indexes) > 0:
                for local_index in local_indexes:
                    extended_index = new_index + local_index
                    indexes.append(extended_index)
            else:
                indexes.append(new_index)
    elif type(value) == dict:
        for k in value.keys():
            new_index = [k]
            local_value = value[k]
            local_indexes = get_indexes(local_value)
            if len(local_indexes) > 0:
                for local_index in local_indexes:
                    extended_index = new_index + local_index
                    indexes.append(extended_index)
            else:
                indexes.append(new_index)
    else:
        indexes.append([])
    return indexes

In [110]:
def drop_index_invalid(samples, indexes):
    valid_filtered_cars = []
    for car in samples:
        car_value = car
        is_valid = True
        for ind in indexes:
            try:
                value_part = get_parameter_value_part(car_value, ind)
            except:
                is_valid = False
                break
        if is_valid:
            valid_filtered_cars.append(car)
    return valid_filtered_cars

In [138]:
def index_to_key(index):
    key = "|".join([str(ind) for ind in index])
    return key
def key_to_index(key):
    ind = key.split("|")
    ind = [k for k in ind if len(str(k)) > 0]
    return ind

In [162]:
def is_equal_jsons(json1, json2):
    is_equal = True
    indexes1 = get_indexes(json1)
    indexes2 = get_indexes(json2)
    
    if len(indexes1) != len(indexes2):
        is_equal - False
    else:
        indexes2_keys = [index_to_key(ind) for ind in indexes2]
        for i in range(len(indexes1)):
            if index_to_key(indexes1[i]) not in indexes2_keys:
                is_equal = False
                break

    if is_equal:
        for ind in indexes1:
            value1 = get_parameter_value_part(json1, ind)
            value2 = get_parameter_value_part(json2, ind)
            if value1 != value2:
                is_equal = False
                break
    return is_equal

In [163]:
def get_valid_values(param, code_json, type_cars):
    parameters = ["Length", "Width", "Height", "WheelWidth", "WheelRadius", "WheelRelativeBiasAlongWidth", "WheelRelativeBiasesAlongLength", "WheelBaseSegmentsSpans", "WheelBaseSegmentsBottomSurfaces", "GapBetweenWheelAndBase", "WheelBaseTopSurface", "BodySegmentsSpans", "BodySegmentsTopSurfaces"]
    filtered_cars = type_cars
    global_indexes = get_indexes(code_json)
    for i, par in enumerate(parameters):
        if i < parameters.index(param):
            sample_value = code_json[par]
            rel_par_indexes = [[ind for ind in index] for index in global_indexes if index[0] == par]
            [index.pop(0) for index in rel_par_indexes]
            full_par_indexes = [[par] + index for index in rel_par_indexes]
            float_rel_par_indexes = [ind for ind in rel_par_indexes if type(get_parameter_value_part(sample_value, ind)) == float]
            str_rel_par_indexes = [ind for ind in rel_par_indexes if type(get_parameter_value_part(sample_value, ind)) == str]
            indexes = get_indexes(sample_value)
            valid_filtered_cars = drop_index_invalid(filtered_cars, full_par_indexes)
            new_filtered_cars = []
            nearest_values = {}
            for ind in float_rel_par_indexes:
                key = index_to_key(ind)
                value = get_parameter_value_part(sample_value, ind)
                values_to_search = [get_parameter_value_part(car[par], ind) for car in valid_filtered_cars]
                nearest_value = get_nearest_value(value, values_to_search)
                nearest_values[key] = nearest_value
            for car in valid_filtered_cars:
                is_car_valid = True
                car_value = car[par]
                for ind in float_rel_par_indexes:
                    sample_value_part = get_parameter_value_part(sample_value, ind)
                    car_value_part = get_parameter_value_part(car_value, ind)
                    if sample_value_part != car_value_part:
                        nearest_value = nearest_values[index_to_key(ind)]
                        if car_value_part != nearest_value:
                            is_car_valid = False
                            break
                for ind in str_rel_par_indexes:
                    sample_value_part = get_parameter_value_part(sample_value, ind)
                    car_value_part = get_parameter_value_part(car_value, ind)
                    if sample_value_part != car_value_part:
                        is_car_valid = False
                        break
                if is_car_valid:
                    new_filtered_cars.append(car)
            filtered_cars = new_filtered_cars 
        else:
            break
    valid_values = [car[param] for car in filtered_cars]
    unique_valid_values = []
    for v in valid_values:
        exists = False
        for u in unique_valid_values:
            if is_equal_jsons(v, u):
                exists = True
                break
        if not exists:
            unique_valid_values.append(v)
    return unique_valid_values

In [194]:
def is_from_distribution(value, valid_values):
    indexes = get_indexes(value)
    ind_valid_values = drop_index_invalid(valid_values, indexes)
    distributions = {}
    for ind in indexes:
        key = index_to_key(ind)
        if key not in distributions:
            distributions[key] = []
        for v in ind_valid_values:
            valid_value_part = get_parameter_value_part(v, ind)
            if value not in distributions[key]:
                distributions[key].append(valid_value_part)
    is_value_valid = True
    for key in distributions.keys():
        ind = key_to_index(key)
        dist = distributions[key]
        value_part = get_parameter_value_part(value, ind)
        if len(dist) == 0:
            is_value_valid = False
            break
        if type(value_part) == float:
            valid_min = min(dist)
            valid_max = max(dist)
            if value_part < valid_min or value_part > valid_max:
                is_value_valid = False
                break
        elif type(value_part) == str:
            if value_part not in dist:
                is_value_valid = False
                break
        else:
            is_value_valid = False
            break
    return is_value_valid

In [193]:
def define_car_type(code_json, codes_by_types):
    valid_types = []
    parameters = ["Length", "Width", "Height", "WheelWidth", "WheelRadius", "WheelRelativeBiasAlongWidth", "WheelRelativeBiasesAlongLength", "WheelBaseSegmentsSpans", "WheelBaseSegmentsBottomSurfaces", "GapBetweenWheelAndBase", "WheelBaseTopSurface", "BodySegmentsSpans", "BodySegmentsTopSurfaces"]
    types = ["sedan", "hatchback", "station wagon", "coupe", "limousine", "SUV", "pickup", "minivan", "van", "truck"]
    for car_type in types:
        type_cars = codes_by_types[car_type]
        passed = True
        for param in parameters:
            valid_values = get_valid_values(param, code_json, type_cars)
            if len(valid_values) == 0:
                passed = False
                break
            from_distribution = is_from_distribution(code_json[param], valid_values)
            if not from_distribution:
                passed = False
                break
        if passed:
            valid_types.append(car_type)
    return valid_types

In [16]:
def get_parameter(class_json, param):
    class_name = class_json[0]
    param_mapping = {
        "MaxHeight":"maxHeight",
        "MinHeight":"minHeight",
        "LeftRounded":"leftRounded",
        "CornerLength":"cornerRelativeLength",
        "TotalLength":"surfaceAbsoluteLength",
        "LeftCorner":"leftRoundedCorner",
        "RightCorner":"rightRoundedCorner"
    }
    if class_name == "Constant":
        param_mapping["MaxHeight"] = "height"
        param_mapping["MinHeight"] = "height"
    return class_json[1][param_mapping[param]]

In [45]:
def sample_with_drop(samples):
    results = [samples]
    if len(samples) > 1:
        for i in range(len(samples)):
            toDrop = [s for s in samples]
            toDrop.pop(i)
            droppedResults = sample_with_drop(toDrop)
            results.extend(droppedResults)
    return results

In [214]:
def num_to_str(num):
    string = ""
    if int(num) - num == 0:
        string = str(int(num))
    else:
        string = str(num)
    return string

In [231]:
def get_prompts_for_code(car, codes_by_types):
    car_types = define_car_type(car, codes_by_types)
    if len(car_types) == 0:
        return []
    allCars = []
    for key in codes_by_types.keys():
        allCars.extend(codes_by_types[key])

    for carType in car_types:
        numParameters = ["Length", "Width", "Height", "WheelRadius"]
        numParameterNames = ["length", "width", "height", "wheel radius"]
        classNumParameterBoundaries = {}
        totalNumParameterBoundaries = {}
        for param in numParameters:
            classNumParameterBoundaries[param] = [1e9, -1e9]
            totalNumParameterBoundaries[param] = [1e9, -1e9]
        for param in numParameters:
            for typeCar in codes_by_types[carType]:
                value = float(typeCar[param])
                if value < classNumParameterBoundaries[param][0]:
                    classNumParameterBoundaries[param][0] = value
                if value > classNumParameterBoundaries[param][1]:
                    classNumParameterBoundaries[param][1] = value
            for typeCar in allCars:
                value = float(typeCar[param])
                if value < totalNumParameterBoundaries[param][0]:
                    totalNumParameterBoundaries[param][0] = value
                if value > totalNumParameterBoundaries[param][1]:
                    totalNumParameterBoundaries[param][1] = value
        basePrompt = "Generate a car with the following characteristics: "
        
        prompts = []
    
        localTexts = []
        prompt = basePrompt
        textVariants = []
        carTypeTextVariants = []
        carTypeTextVariants.append(prompt)
        carTypeTextVariants.append(prompt + f"body type: {carType};")
        typeParameterDescriptions = []
        description = f"{len(car['WheelRelativeBiasesAlongLength'])} wheelsets"
        typeParameterDescriptions.append(description)
        if carType == "sedan" or carType == "coupe":
            typeParameterDescriptions.append("normal size bonnet")
            if car["BodySegmentsSpans"][2][0] == 0.8:
                typeParameterDescriptions.append("normal size boot")
            elif car["BodySegmentsSpans"][2][0] == 0.9:
                typeParameterDescriptions.append("small boot")
        elif carType == "hatchback" or carType == "SUV":
            typeParameterDescriptions.append("normal size bonnet")
            typeParameterDescriptions.append("normal size boot")
            typeParameterDescriptions.append("door at the rear")
        elif carType == "station wagon":
            typeParameterDescriptions.append("normal size bonnet")
            typeParameterDescriptions.append("large body")
            typeParameterDescriptions.append("large boot")
            typeParameterDescriptions.append("door at the rear")
        elif carType == "limousine":
            typeParameterDescriptions.append("normal size bonnet")
            if car["BodySegmentsSpans"][2][0] == 0.8:
                typeParameterDescriptions.append("normal size boot")
            elif car["BodySegmentsSpans"][2][0] == 0.9:
                typeParameterDescriptions.append("small boot")
            typeParameterDescriptions.append("very long body")
        elif carType == "pickup":
            typeParameterDescriptions.append("normal size bonnet")
            typeParameterDescriptions.append("large boot")
        elif carType == "minivan":
            typeParameterDescriptions.append("small bonnet")
            typeParameterDescriptions.append("large body")
            typeParameterDescriptions.append("no boot")
        elif carType == "van":
            typeParameterDescriptions.append("normal size bonnet")
            if float(get_parameter(car["BodySegmentsTopSurfaces"][2][1], "MaxHeight")) > float(get_parameter(car["BodySegmentsTopSurfaces"][1][1], "MaxHeight")):
                typeParameterDescriptions.append("1 cargo container")
            else:
                typeParameterDescriptions.append("1 flatbed")
        elif carType == "truck":
            if float(get_parameter(car["BodySegmentsTopSurfaces"][0][1], "MaxHeight")) > float(car["Height"]) * 0.6:
                typeParameterDescriptions.append("no bonnet")
            elif float(get_parameter(car["BodySegmentsTopSurfaces"][0][1], "MaxHeight")) > float(car["Height"]) * 0.6:
                typeParameterDescriptions.append("large bonnet")
            fairing = False
            for surf in car["BodySegmentsTopSurfaces"]:
                if surf[0] == "TotalRounded" and get_parameter(surf[1], "MaxHeight") == car["Height"]:
                    fairing = True
                    break
            if fairing:
                typeParameterDescriptions.append("roof fairing with sleeper cab")
            cargoCount = 0
            for i in range(len(car["BodySegmentsSpans"])):
                span = car["BodySegmentsSpans"][i]
                if span[1] - span[0] > 0.17:
                    cargoCount = len(car["BodySegmentsTopSurfaces"]) - i
                    break
            if cargoCount == 1:
                typeParameterDescriptions.append("1 cargo container")
            elif cargoCount > 1:
                typeParameterDescriptions.append(f"{cargoCount} cargo containers")
        descriptionSamples = sample_with_drop(typeParameterDescriptions)
        for sample in descriptionSamples:
            carTypeTextVariants.append(prompt + ";".join(sample))
        for carTypePrompt in carTypeTextVariants:
            textVariants.append(carTypePrompt)
            masks = []
            for j in range(2 ** len(numParameters)):
                mask = [None] * len(numParameters)
                div = j
                ind = len(numParameters) - 1
                while div > 0:
                    mask[ind] = div % 2
                    ind -= 1
                    div = int(div/2)
                for k in range(ind + 1):
                    mask[k] = 0
                masks.append(mask)
            for mask in masks:
                numParameterDescriptions = []
                for k in range(len(numParameters)):
                    info = numParameters[k]
                    value = float(car[info])
                    if mask[k] == 0:
                        numParameterDescriptions.append(f"{numParameterNames[k]}: {num_to_str(value)} cm")
                    else:
                        desc = ""
                        minv = 0
                        maxv = 0
                        if "body type:" in carTypePrompt:
                            minv = classNumParameterBoundaries[info][0]
                            maxv = classNumParameterBoundaries[info][1]
                        else:
                            minv = totalNumParameterBoundaries[info][0]
                            maxv = totalNumParameterBoundaries[info][1]
                        leftMedium = minv + (maxv - minv) / 3
                        rightMedium = minv + (maxv - minv) * 2 / 3
                        if value <= leftMedium:
                            desc = "small"
                        elif value >= rightMedium:
                            desc = "big"
                        else:
                            desc = "medium"
                        numParameterDescriptions.append(f"{numParameterNames[k]} is {desc}")
                numParameterDescriptionSamples = sample_with_drop(numParameterDescriptions)
                for sample in numParameterDescriptionSamples:
                    textVariants.append(f"{carTypePrompt}; {'; '.join(sample)}")
        corrected_prompts = []
        pattern = "(?<=;|:)\s*?;"
        for text in textVariants:
            cor = re.sub(pattern, "", text)
            corrected_prompts.append(cor)
        finalPrompts = []
        for text in corrected_prompts:
            if text not in finalPrompts:
                finalPrompts.append(text)
        prompts.extend(finalPrompts)
    return prompts

In [201]:
text = "Generate a car with the following characteristics: body type: sedan;; length: 250.0 cm; width: 167.0 cm; height: 150.0 cm; wheel radius: 25.0 cm"
pattern = "(?<=;|:)\s*?;"
re.sub(pattern, "", text)

'Generate a car with the following characteristics: body type: sedan; length: 250.0 cm; width: 167.0 cm; height: 150.0 cm; wheel radius: 25.0 cm'

## Scores

In [19]:
#measures the part of generations wich code can be extracted from
def extraction_score(generations):
    score = 0
    pattern = "(?<=```csharp)(?:.|\s)+?(?=```)"
    for gen in tqdm(generations):
        code = re.findall(pattern, gen)
        if len(code) != 0:
            code = code[0]
            score += 1
    score = score / len(generations)
    return score

In [20]:
#measures the part of generations wich code can be transformed to json. Generations should pass the extraction score before.
def compilation_score(codes):
    score = 0
    for code in tqdm(codes):
        try:
            code_json = code_to_json(code)
            score += 1
        except:
            continue
    score = score / len(codes)
    return score

In [229]:
#measures the part of generations wich compilled jsons can be transformed to templated text inputs correctly
def reconstruction_score(jsons, descriptions, codes_by_types):
    score = 0
    total_samples = 0
    for ind, code_json in tqdm(enumerate(jsons), total = len(jsons), desc="Computing score"):
        desc = descriptions[ind]
        pattern = "(?<=;|:)\s*?;"
        desc = re.sub(pattern, "", desc)
        total_samples += 1
        prompts = get_prompts_for_code(code_json, codes_by_types)
        if desc in prompts:
            score += 1
    score = score / total_samples
    return score

## Evaluation

In [16]:
print("""{\"rewritten request\":\"Craft a lengthy vehicle, approximately 7 meters in length, with a medium width and a compact hood. It should have two sets of wheels, each with a small radius.\",\n\"explanation\":\"I started with 'Craft a lengthy vehicle' to emphasize the 'very long body'. I mentioned the length here: 'approximately 7 meters in length'. 'Medium width' is mentioned here: 'with a medium width'. 'Compact hood' refers to 'normal size bonnet'. I wrote about the wheel sets here: 'It should have two sets of wheels', and about the wheel radius here: 'each with a small radius'.\"}""")

{"rewritten request":"Craft a lengthy vehicle, approximately 7 meters in length, with a medium width and a compact hood. It should have two sets of wheels, each with a small radius.",
"explanation":"I started with 'Craft a lengthy vehicle' to emphasize the 'very long body'. I mentioned the length here: 'approximately 7 meters in length'. 'Medium width' is mentioned here: 'with a medium width'. 'Compact hood' refers to 'normal size bonnet'. I wrote about the wheel sets here: 'It should have two sets of wheels', and about the wheel radius here: 'each with a small radius'."}


In [22]:
!unzip code_by_types.zip

Archive:  code_by_types.zip
   creating: code_by_types/
   creating: code_by_types/Coupe/
  inflating: code_by_types/Coupe/Code_0.cs  
  inflating: code_by_types/Coupe/Code_1.cs  
  inflating: code_by_types/Coupe/Code_10.cs  
  inflating: code_by_types/Coupe/Code_100.cs  
  inflating: code_by_types/Coupe/Code_101.cs  
  inflating: code_by_types/Coupe/Code_102.cs  
  inflating: code_by_types/Coupe/Code_103.cs  
  inflating: code_by_types/Coupe/Code_104.cs  
  inflating: code_by_types/Coupe/Code_105.cs  
  inflating: code_by_types/Coupe/Code_106.cs  
  inflating: code_by_types/Coupe/Code_107.cs  
  inflating: code_by_types/Coupe/Code_108.cs  
  inflating: code_by_types/Coupe/Code_109.cs  
  inflating: code_by_types/Coupe/Code_11.cs  
  inflating: code_by_types/Coupe/Code_110.cs  
  inflating: code_by_types/Coupe/Code_111.cs  
  inflating: code_by_types/Coupe/Code_112.cs  
  inflating: code_by_types/Coupe/Code_113.cs  
  inflating: code_by_types/Coupe/Code_114.cs  
  inflating: code_by_ty

In [8]:
dataset = load_dataset(dataset_name)

with open("train_nl.json", "r") as f:
    data = json.load(f)
small_dataset = {"input":[], "natural_input":[], "output":[], "text": []}
keys = [int(k) for k in data.keys()]
sorted_keys = sorted(keys)
for ind in tqdm(sorted_keys):
    natural_input = ""
    try:
        natural_input = json.loads(data[str(ind)])["rewritten request"]
    except:
        continue
    if natural_input != "":
        row = dataset["train"][ind]
        small_dataset["input"].append(row["input"])
        small_dataset["output"].append(row["output"])
        small_dataset["text"].append("")
        small_dataset["natural_input"].append(natural_input)
small_dataset = Dataset.from_dict(small_dataset)
print(f"Dataset length: {len(small_dataset)}")
small_dataset = small_dataset.train_test_split(test_size=0.05, seed=42)

  0%|          | 0/13462 [00:00<?, ?it/s]

Dataset length: 13442


In [9]:
small_dataset["test"]["natural_input"][232]

'Craft a compact, two-wheeled vehicle. It should measure 1.67 meters in width and 1.35 meters in height, with small-radius wheels.'

In [10]:
with open("generations.json", "r") as f:
    generations = json.load(f)
eval_dataset = {"input":[], "output":[], "natural_input":[], "generation":[]}
for key in generations.keys():
    ind = int(key)
    row = small_dataset["test"][ind]
    gen = generations[key]
    eval_dataset["generation"].append(gen)
    eval_dataset["input"].append(row["input"])
    eval_dataset["output"].append(row["output"])
    eval_dataset["natural_input"].append(row["natural_input"])
eval_dataset = Dataset.from_dict(eval_dataset)

In [15]:
ind = 300
print(eval_dataset["natural_input"][ind])
print(eval_dataset["generation"][ind])

Craft a compact vehicle with a small hood and a spacious body, devoid of a boot. It should have two wheelsets, each with a radius of 40 cm, and a width of 1.67 meters.
Here's the C# class for the car you described, with the parameters filled according to the user's request:

```csharp
using System.Collections.Generic;
using System;
using System.Collections;

namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 150f;

            WheelWidth = 20f;
            WheelRadius = 40f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 20f) };
            WheelBaseTopSurface = new Constant(height: 80f);
            GapBet

In [68]:
extraction = extraction_score(eval_dataset["generation"])
print(f"Extraction score: {extraction}")

  0%|          | 0/625 [00:00<?, ?it/s]

Extraction score: 1.0


In [69]:
codes = extract_codes(eval_dataset["generation"])

  0%|          | 0/625 [00:00<?, ?it/s]

In [78]:
compilation = compilation_score(codes)
print(f"Compilation score: {compilation}")

  0%|          | 0/625 [00:00<?, ?it/s]

Compilation score: 0.9952


In [79]:
inds, jsons = transform_codes_to_jsons(codes)

  0%|          | 0/625 [00:00<?, ?it/s]

In [80]:
descriptions = [eval_dataset[ind]["input"] for ind in inds]

In [81]:
path = "code_by_types"
codes_dict = {}
for d in tqdm(os.listdir(path), desc = "Preparing data"):
    codes_path = os.path.join(path, d)
    codes_dict[d] = []
    for name in os.listdir(codes_path):
        with open(os.path.join(codes_path, name), "r") as f:
            code = f.read()
        code_json = code_to_json(code)
        codes_dict[d].append(code_json)
type_mapping = {
    "Coupe": "coupe",
    "Hatchback":"hatchback",
    "Limousine":"limousine",
    "Minivan":"minivan",
    "Pickup":"pickup",
    "Sedan":"sedan",
    "StationWagon":"station wagon",
    "SUV":"SUV",
    "Truck":"truck",
    "Van":"van",
}
codes_by_types = {}
for key in codes_dict.keys():
    codes_by_types[type_mapping[key]] = codes_dict[key]

Preparing data:   0%|          | 0/10 [00:00<?, ?it/s]

In [230]:
reconstruction = reconstruction_score(jsons, descriptions, codes_by_types)
print(f"Reconstruction score: {reconstruction}")

Computing score:   0%|          | 0/622 [00:00<?, ?it/s]

['sedan']
['sedan']
['sedan']
['sedan']
['sedan']
['sedan']
['sedan']
Reconstruction score: 0.006430868167202572


# Bot

In [1]:
def get_inference_text(prompt, text, tokenizer):
  messages = [
      {"role": "system", "content": prompt},
      {"role": "user", "content": text}
  ]
  input_text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  return input_text

In [2]:
def generate(model, tokenizer, generation_config, inp):
  tokenized_prompt = tokenizer(inp, return_tensors='pt', padding=True, truncation=True)
  tokenized_prompt_ids = tokenized_prompt["input_ids"].cuda()
  tokenized_prompt_mask = tokenized_prompt["attention_mask"].cuda()
  with torch.inference_mode():
      output = model.generate(**{"input_ids":tokenized_prompt_ids, "attention_mask":tokenized_prompt_mask, "generation_config":generation_config}).detach().cpu()
  decoded = []
  for i in range(output.shape[0]):
    ans = tokenizer.decode(output[i][len(tokenized_prompt[0]):], skip_special_tokens=True)
    decoded.append(ans)
  return decoded

In [3]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from peft import PeftModel
import torch
import re

In [4]:
text2code_model_name = "mistralai/Mistral-Nemo-Instruct-2407"
text2code_tuned_model_name = "evgmaslov/Mistral-Nemo-Instruct-2407-cars"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(text2code_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.chat_template = """{%- if messages[0][\"role\"] == \"system\" %}\n    {%- set system_message = messages[0][\"content\"] %}\n    {%- set loop_messages = messages[1:] %}\n{%- else %}\n    {%- set loop_messages = messages %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n{%- set user_messages = loop_messages | selectattr(\"role\", \"equalto\", \"user\") | list %}\n\n{#- This block checks for alternating user/assistant messages, skipping tool calling messages #}\n{%- set ns = namespace() %}\n{%- set ns.index = 0 %}\n{%- for message in loop_messages %}\n    {%- if not (message.role == \"tool\" or message.role == \"tool_results\" or (message.tool_calls is defined and message.tool_calls is not none)) %}\n        {%- if (message[\"role\"] == \"user\") != (ns.index % 2 == 0) %}\n            {{- raise_exception(\"After the optional system message, conversation roles must alternate user/assistant/user/assistant/...\") }}\n        {%- endif %}\n        {%- set ns.index = ns.index + 1 %}\n    {%- endif %}\n{%- endfor %}\n\n{{- bos_token }}\n{%- for message in loop_messages %}\n    {%- if message[\"role\"] == \"user\" %}\n        {%- if tools is not none and (message == user_messages[-1]) %}\n            {{- \"[AVAILABLE_TOOLS][\" }}\n            {%- for tool in tools %}\n                {%- set tool = tool.function %}\n                {{- '{\"type\": \"function\", \"function\": {' }}\n                {%- for key, val in tool.items() if key != \"return\" %}\n                    {%- if val is string %}\n                        {{- '\"' + key + '\": \"' + val + '\"' }}\n                    {%- else %}\n                        {{- '\"' + key + '\": ' + val|tojson }}\n                    {%- endif %}\n                    {%- if not loop.last %}\n                        {{- \", \" }}\n                    {%- endif %}\n                {%- endfor %}\n                {{- \"}}\" }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- else %}\n                    {{- \"]\" }}\n                {%- endif %}\n            {%- endfor %}\n            {{- \"[/AVAILABLE_TOOLS]\" }}\n            {%- endif %}\n        {%- if system_message is defined %}\n            {{- \"[INST]\" + system_message + \"\\n\\n\" + message[\"content\"] + \"[/INST]\" }}\n        {%- else %}\n            {{- \"[INST]\" + message[\"content\"] + \"[/INST]\" }}\n        {%- endif %}\n    {%- elif (message.tool_calls is defined and message.tool_calls is not none) %}\n        {{- \"[TOOL_CALLS][\" }}\n        {%- for tool_call in message.tool_calls %}\n            {%- set out = tool_call.function|tojson %}\n            {{- out[:-1] }}\n            {%- if not tool_call.id is defined or tool_call.id|length != 9 %}\n                {{- raise_exception(\"Tool call IDs should be alphanumeric strings with length 9!\") }}\n            {%- endif %}\n            {{- ', \"id\": \"' + tool_call.id + '\"}' }}\n            {%- if not loop.last %}\n                {{- \", \" }}\n            {%- else %}\n                {{- \"]\" + eos_token }}\n            {%- endif %}\n        {%- endfor %}\n    {%- elif message[\"role\"] == \"assistant\" %}\n        {{- message[\"content\"] + eos_token}}\n    {%- elif message[\"role\"] == \"tool_results\" or message[\"role\"] == \"tool\" %}\n        {%- if message.content is defined and message.content.content is defined %}\n            {%- set content = message.content.content %}\n        {%- else %}\n            {%- set content = message.content %}\n        {%- endif %}\n        {{- '[TOOL_RESULTS]{\"content\": ' + content|string + \", \" }}\n        {%- if not message.tool_call_id is defined or message.tool_call_id|length != 9 %}\n            {{- raise_exception(\"Tool call IDs should be alphanumeric strings with length 9!\") }}\n        {%- endif %}\n        {{- '\"call_id\": \"' + message.tool_call_id + '\"}[/TOOL_RESULTS]' }}\n    {%- else %}\n        {{- raise_exception(\"Only user and assistant roles are supported, with the exception of an initial optional system message!\") }}\n    {%- endif %}\n{%- endfor %}\n"""

In [6]:
quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_compute_dtype=torch.float16,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_use_double_quant=True)

In [7]:
model = AutoModelForCausalLM.from_pretrained(text2code_model_name,
                                             quantization_config=quant_config,
                                             device_map='auto',
                                  low_cpu_mem_usage=True, offload_state_dict=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
model = PeftModel.from_pretrained(model, text2code_tuned_model_name)

In [9]:
prompt = """You are an experienced engineer who designs cars. You help people get their dream car. You receive a request to create a car, which describes its shape, size, and other characteristics. To create a car, you use a C# class of the following structure:
public Car()
{
    Length = ...;
    Width = ...;
    Height = ...;

    WheelWidth = ...;
    WheelRadius = ...;

    WheelRelativeBiasAlongWidth = ...;
    WheelRelativeBiasesAlongLength = ...;

    WheelBaseSegmentsSpans = ...;
    WheelBaseSegmentsBottomSurfaces = ...;
    WheelBaseTopSurface = ...;
    GapBetweenWheelAndBase = ...;

    BodySegmentsSpans = ...;
    BodySegmentsTopSurfaces = ...;
}.
Here is a description of its parameters. "Length" is the length of the car in centimeters, this is a float type parameter. "Width" is the length of the car in centimeters, this is a float type parameter. "Height" is the length of the car in centimeters, this is a float type parameter. "WheelWidth" is the width of each wheel in centimeters, this is a float type parameter. "WheelRadius" is the radius of each wheel in centimeters, this is a float type parameter. "WheelRelativeBiasAlongWidth" is a parameter that determines how far the wheel is shifted into the car body. For example, if WheelRelativeBiasAlongWidth = 0.1, then the wheel is shifted into the car by a distance equal to 10% of its width. This parameter is of float type. "WheelRelativeBiasesAlongLength" is a parameter that contains a list of numbers in floating point format. The number of values in the list is equal to the number of wheel pairs the car has. Each value is equal to the relative offset of a wheel pair relative to the front of the car. For example, if WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f }, then the car has two wheel pairs, one offset by 20% of the length, and the other by 80%. The "WheelBaseSegmentsSpans" parameter describes the arrangement of the parts of the car's floor. The floor of a car is the frame on which its wheels are mounted. Usually a car has one frame, but a truck can have 2 or more frames. This parameter contains a list where each element is a range of lengths in which the floor of the car is located. The range is described by a list where the first element is the start of the range and the second element is the end of the range. The WheelBaseSegmentsBottomSurfaces parameter describes the shape of the bottom of the car. If the bottom of the car consists of several planes, this parameter will contain several planes. This parameter contains as many planes as there are ranges specified in the "WheelBaseSegmentsSpans" parameter, because each plane has its own range. This parameter can be used to adjust the ground clearance of the car. The BodySegmentsSpans parameter contains a list of length ranges in which the vehicle body components are located. The "BodySegmentsTopSurfaces" parameter contains a list of the components of the car body. The number of components of the car is equal to the number of ranges in the "BodySegmentsSpans" parameter, because each component has its own range. Each component of the car is described by the shape of its surface. The shape can be specified by one of three classes: Constant (plain shape), CornerRounded (plain shape with rounded corners) and TotalRounded (rounded shape).
The user will ask you to generate a car and will provide you with a description of its shape and dimensions. You need to extract from his request the information needed to fill in the C# class parameters described above. Write C# class to generate a car, fill in its parameters according to the user's request. Here are some examples of what you need to do:
Example 1
user request: Design a car with a standard-sized hood and trunk. It should be 1.67 meters wide and have wheels with a radius of 40 cm.
car: using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 150f;

            WheelWidth = 20f;
            WheelRadius = 40f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 13f) };
            WheelBaseTopSurface = new Constant(height: 90f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.3f }, new List<float>(){ 0.3f, 0.8f }, new List<float>(){ 0.8f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new Constant(height: 105f), new Constant(height: 150f), new CornerRounded(minHeight: 90f, maxHeight: 105f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 49.999996f, leftCornerRounded: false, rightCornerRounded: true) };
        }
    }

}
Example 2:
user request: Design a car with two wheelsets and a standard-sized hood. It should be 1.67 meters wide.
car: using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 150f;

            WheelWidth = 20f;
            WheelRadius = 25f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 18f) };
            WheelBaseTopSurface = new Constant(height: 75f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.3f }, new List<float>(){ 0.3f, 0.9f }, new List<float>(){ 0.9f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new TotalRounded(minHeight: 75f, maxHeight: 90f, leftRounded: true), new Constant(height: 150f), new CornerRounded(minHeight: 75f, maxHeight: 90f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 25.000006f, leftCornerRounded: false, rightCornerRounded: true) };
        }
    }

}
Example 3: 
user request: Design a compact sedan. It should be short and narrow, with a small height.
car: using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 135f;

            WheelWidth = 20f;
            WheelRadius = 25f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 13f) };
            WheelBaseTopSurface = new Constant(height: 67.5f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.2f }, new List<float>(){ 0.2f, 0.8f }, new List<float>(){ 0.8f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new CornerRounded(minHeight: 67.5f, maxHeight: 94.5f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 50f, leftCornerRounded: true, rightCornerRounded: false), new CornerRounded(minHeight: 94.5f, maxHeight: 135f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 150f, leftCornerRounded: true, rightCornerRounded: true), new CornerRounded(minHeight: 67.5f, maxHeight: 94.5f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 49.999996f, leftCornerRounded: false, rightCornerRounded: true) };
        }
    }

}
Example 4:
user request: Design a car with two wheelsets, a standard-sized hood, and a height of 1.35 meters. Ensure the wheel radius is large.
car: using System.Collections.Generic;
using System;
using System.Collections;
namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 167f;
            Height = 135f;

            WheelWidth = 20f;
            WheelRadius = 40f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 18f) };
            WheelBaseTopSurface = new Constant(height: 94.5f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.2f }, new List<float>(){ 0.2f, 0.8f }, new List<float>(){ 0.8f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new TotalRounded(minHeight: 94.5f, maxHeight: 108f, leftRounded: true), new CornerRounded(minHeight: 108f, maxHeight: 135f, cornerRelativeLength: 0.2f, surfaceAbsoluteLength: 150f, leftCornerRounded: true, rightCornerRounded: true), new TotalRounded(minHeight: 94.5f, maxHeight: 108f, leftRounded: false) };
        }
    }

}"""

In [10]:
user_prompt = "Craft a car with the width of 165 cm and wheel radius of 30 cm."
gen_config = GenerationConfig.from_pretrained(text2code_model_name)
gen_config.max_length = 4096
output = generate(model, tokenizer, gen_config, get_inference_text(prompt, user_prompt, tokenizer))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [11]:
pattern = "(?<=```csharp)(?:.|\s)+?(?=```)"
code = re.findall(pattern, output[0])
if len(code) > 0:
    code = code[0]
else:
    code = ""

In [12]:
print(code)


using System.Collections.Generic;
using System;
using System.Collections;

namespace GenerativeDesign.Cars
{
    public class Car : CarBase
    {
        public Car()
        {
            Length = 250f;
            Width = 165f;
            Height = 150f;

            WheelWidth = 20f;
            WheelRadius = 30f;
            WheelRelativeBiasAlongWidth = 0.1f;
            WheelRelativeBiasesAlongLength = new List<float>() { 0.2f, 0.8f };

            WheelBaseSegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 1f } };
            WheelBaseSegmentsBottomSurfaces = new List<Line>() { new Constant(height: 22f) };
            WheelBaseTopSurface = new Constant(height: 85f);
            GapBetweenWheelAndBase = 3f;

            BodySegmentsSpans = new List<List<float>>() { new List<float>(){ 0f, 0.3f }, new List<float>(){ 0.3f, 0.9f }, new List<float>(){ 0.9f, 1f } };
            BodySegmentsTopSurfaces = new List<Line>() { new TotalRounded(minHeight: 85f, maxHeight: 100f,